# Graph Neural Networks to Predict Protein Isoelectric Points

### This notebook will become available on github as soon as my preprint [1] is available online, here: <a href="https://chemrxiv.org/engage/chemrxiv/article-details/6399e788ff4651008f2a4e2b">TB 2022</a>. 
### This notebook presents the code used to train graph neural networks on prediction of protein isoelectric points, using experimental datasets curated and made available within the public domain by L. P. Kozlowski <a href="www.ipc2-isoelectric-point.org">(see IPC 2) [2]</a>. As Spektral [3] is used in conjunction with Tensorflow and Keras, we need to import those (and several other) libraries; this notebook is Google Colab–ready, i.e., I provide commands such as the next cell to enable working on Colab. The files provided must obviously be uploaded to the content (root) directory, though.



***

In [ ]:
#Install Spektral if working on Google Colab
!pip install spektral
#Let's install Bio as well. just because...
!pip install Bio
from Bio import SeqIO

In [ ]:
#Import everything else
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf 
keras = tf.keras 
import os
import math
import scipy.sparse as spar
import spektral as spk
from spektral.data import Dataset, DisjointLoader, Graph, SingleLoader
from spektral.transforms.normalize_adj import NormalizeAdj as NormAdj
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError, Huber
import csv


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

***

#### We need to apply the Henderson-Hasselbalch equation differently for basic and acidic AAs. Cysteine,  asprtic acid, glutamic acid and tyrosine are acidic. Histidine, lysine and arginine are basic. Positions 8 and 9 in the following array refer to the C and N termini, the former of which is acidic.



***

In [ ]:
IsAcid = [True,True,True,False,False,False,True, True, False]

***

#### Here are some pKa values to start experimenting with, taken from [4]. These will come in handy when calculating baseline isoelectric point (pI) values.



***

In [ ]:
# pKa constants taken from [4], cf. AA_List
pKa_List = [8.3, 3.9, 4.3, 6, 10.5, 12.5, 10.1, 2.4, 9.6]

In [ ]:
AA_List = ['C','D','E','H','K','R','Y'];
AA_List_Full = ['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V']

***

####  Let's create a function to prepare datasets, reading data from a file with an 'exp_pI' column, containing the experimental pI values, and a string of amino acids (AAs) for each AA sequence. 
####  The Calc_Charge function calculates the charge (in atomic units) on a protein using the number of residues of the nine AAs of residues of interest mentioned before, at single pH value, and utilizing nine pKa values used as input. Note that the IsAcid values tell the calculation whether the protonated form of the AA has a charge of 0 (True, acidic AAs) or +1 (False, basic AAs). Everything else is just a simple manipulation of the Henderson-Hasselbalch equation. We will need this function to get the baseline, zeroth-approximation pI values.   
#### pI_HH solves numerically for the root of the Calc_Charge function. It would be extremely simple to calculate the charge for pH from 0 to 14 using 0.0001 pH unit steps; our later calculations are much heavier. Having seen attempts to leverage inefficient methods to speed up such simple numerical calculations, however, I deemed it appropriate to show how to do it properly with a quick and dirty Newton method implementation; to avoid the "Newton Method's Spiral of Death", the largest pH step allowed is 0.25 pH units. The root of the function is found when the Newton's method step is below 0.0025 pH units, and the charge on the protein is under 10**-20. That's a pretty small number.  
####  The variables returned are the labels (difference between exp_pI and calculated baseline pI values), the numbered AA sequnces, a scalar telling us how many AA sequences are in the dataset (nu_samples) and an array containing the lengths of all AA sequences (Nodes). 

In [ ]:
def prep_dataset(IsAcid,pKa_List,AA_List, AA_List_Full,file_path,DirectInput=None):
    sequences = []
    Nodes = []
    if DirectInput:
        Nodes = [len(DirectInput)]
        exp_pI = [7]
        nu_samples = len(exp_pI)
        Nu_AA1 = np.zeros((nu_samples, 7))
        Nu_AA2 = np.ones((nu_samples, 2))
        Nu_AA = np.concatenate((Nu_AA1, Nu_AA2), axis=1)
        s = [20]*Nodes
        for jj in range(20):
            positions = [q.start() for q in re.finditer(AA_List_Full[jj], DirectInput)]
            for pos in positions:
                s[int(pos)] = int(jj)
        sequences.append(s)
        for jj in range(0,7):
            Nu_AA[j,jj] = DirectInput.count(AA_List[jj])
    elif file_path.lower().endswith('csv'):
        df = pd.read_csv(file_path)
        exp_pI = df.iloc[:,0]; #or df['exp_pI']
        nu_samples = len(exp_pI)
        Nu_AA1 = np.zeros((nu_samples, 7))
        Nu_AA2 = np.ones((nu_samples, 2))
        Nu_AA = np.concatenate((Nu_AA1, Nu_AA2), axis=1)
        for j in range(0,nu_samples): 
            one_sequence = df.loc[j][1]
            for jj in range(0,7):                
                Nu_AA[j,jj] = one_sequence.count(AA_List[jj])
            s = [20]*len(one_sequence)
            for jk in range(20):
                positions = [q.start() for q in re.finditer(AA_List_Full[jk], one_sequence)]
                for pos in positions:
                    s[int(pos)] = int(jk)
            sequences.append(s)
            Nodes.append(int(len(one_sequence)))
    elif file_path.lower().endswith('fasta'):
        fasta_sequences = SeqIO.parse(open(file_path),'fasta')
        nu_samples = len([0 for fasta in fasta_sequences])
        exp_pI = [7]*nu_samples
        Nu_AA1 = np.zeros((nu_samples, 7))
        Nu_AA2 = np.ones((nu_samples, 2))
        Nu_AA = np.concatenate((Nu_AA1, Nu_AA2), axis=1)
        fasta_sequences = SeqIO.parse(open(file_path),'fasta') #and again, 'cos it's SSSSSSSTUPID!
        j = 0
        for fasta in fasta_sequences:
            one_sequence = str(fasta.seq)
            for jj in range(0,7):
                Nu_AA[j,jj] = one_sequence.count(AA_List[jj])
            s = [20]*len(one_sequence)
            for jk in range(20):
                positions = [q.start() for q in re.finditer(AA_List_Full[jk], one_sequence)]
                for pos in positions:
                    s[int(pos)] = int(jk)
            sequences.append(s)
            Nodes.append(int(len(one_sequence)))
            j+=1 
                
                    
            
    def Calc_Charge(pH, IsAcid, pKa_List, Nu_AA):
        Charge = 0
        dq_dx = 0
        for j in range(0,len(Nu_AA)):
            Ratio = 10**(pKa_List[j]-pH)
            Frac = Ratio/(1+Ratio)
            Charge = Charge + Nu_AA[j]*(-IsAcid[j] + Frac)
            dq_dx = dq_dx - math.log(10,math.exp(1))*Nu_AA[j]*Ratio*(1+Ratio)**-2

        return Charge, dq_dx
    
    def pI_HH(exp_pI, IsAcid, pKa_List, Nu_AA):
        x = exp_pI + 1
        corrx = 1 #Ensure "correction" onto exp_pI on the first step
        while abs(corrx) >= .0025: 
            x = x - corrx
            Charge, dq_dx = Calc_Charge(x, IsAcid, pKa_List, Nu_AA)
        
            if abs(dq_dx) > 10**-20:
                corrx = Charge/dq_dx
                corrx = np.sign(corrx)*min(.25,abs(corrx))
            else:
                corrx = 0
            
        return x      

    
    C = np.zeros(0)
    for j in range(0,nu_samples):
        c = pI_HH(exp_pI[j], IsAcid, pKa_List, Nu_AA[j])
        C = np.append(C,[c],axis=0)
    
    labels = exp_pI - C
    return sequences, Nodes, nu_samples, labels, C
    

             


***

#### Let's prepare a combined dataset of the peptide and protein training sets. We'll save it under Combined_Training.csv.

In [ ]:
df = pd.read_csv('/content/IPC2_protein_75.csv')
df1 = pd.read_csv('/content/IPC2_peptide_75.csv')
exp_pI = df.iloc[:,0]; #or df['exp_pI']
exp_pI1 = df1.iloc[:,0];
S = df.iloc[:,1]; #or df['sequence']
S1 = df1.iloc[:,1]; 
exp_pI = np.concatenate((exp_pI,exp_pI1),axis=0)
S = np.concatenate((S,S1),axis=0)
De = {'exp_pI': exp_pI, 'sequence': S}
df = pd.DataFrame(data=De)
df.to_csv('Combined_Training.csv',index=False)

In [ ]:
sequences, Nodes, nu_samples, labels, C = prep_dataset(IsAcid,pKa_List,AA_List, AA_List_Full,'/content/Combined_Training.csv')

***

#### Let's load the physical descriptors. They are already normalized for our convenience; the original values are given in Ref. [1],[5]–[7]. For reference, these descripotors are ordered as ionization energy, electron affinity, polarizability, dipole moment, hydrophobicity, volume, and alpha-helix and beta-sheet propensity values; see original publications for more details. The first twenty entries correspond to our AA_List_Full array, and the twenty-first value is an average, reserved for unknown AAs in the sequence.   

***

In [ ]:
col21_28 = pd.read_csv('/content/X21to28.csv',header=None)

***

#### The following builds the dataset used by Spektral. Reading the original documentation from [3] will come in handy here; note that we allow for three variations: 21 node features given as a one-hot encoding of the AAs, eight node features corresponding to the physical descriptors, and a combination of those.

***

In [ ]:
################################################################################
# Load data
################################################################################
class MyDataset(Dataset):

    def __init__(self, nu_samples, n_AA, **kwargs): #n_nodes, sequences, labels, ):
        self.nu_samples = nu_samples
        self.n_AA = n_AA
        #self.n_nodes = n_nodes
        #self.sequences = sequences
        for key, value in kwargs.items():
            self.key = value
        super().__init__(**kwargs)

    def read(self):
        def make_graph(ZZ):
            n = self.Nodes[ZZ]
            AA = self.sequences[ZZ]
            exp_values = self.col21_28
            JD = self.Just_Descriptors
            # Node features
            if exp_values is None:
                x = np.zeros((n, self.n_AA))
                x[np.arange(n), AA] = 1
            else:
                x = np.zeros((n, self.n_AA+8))
                for j in range(self.n_AA):
                    s = [i for i,Y in enumerate(AA) if Y==j]   
                    if s:
                        x[s, 21:] = [exp_values.iloc[:,j] for _ in range(len(s))]
                if JD:
                    x = np.delete(x,np.s_[:21],1)
                
           
                
                
        
            # Edges
            a = np.eye(n)
            a[np.arange(n-1),np.arange(1,n)] = 1
            a[np.arange(1,n),np.arange(n-1)] = 1
            a = spar.csr_matrix(a)

            # Labels
            y = self.labels[ZZ]

            return Graph(x=x, a=a, y=y)

        # We must return a list of Graph objects
        return [make_graph(ZZ) for ZZ in range(self.nu_samples)] 
    

### Let's create the 29-feature dataset. I'll be showing only the (smaller) protein dataset; later, I'll show the work on the combined (protein+peptide) dataset. It is trivial to do the work on the separate peptide dataset.

In [ ]:
dataset = MyDataset(nu_samples,n_AA=21,Nodes=Nodes,sequences=sequences,labels=labels,
                    col21_28=col21_28,Just_Descriptors=False,
                    transforms=NormAdj())

***

In [ ]:
#We want to always use the same training and validation sets
tf.random.set_seed(1)

In [ ]:
#To divide into 85% training and 15% validation; we don't touch the testing set
#In this here town we like our batch size to be kept at 32
idxs = np.random.permutation(len(dataset))  # Random split
split = int(0.85 * len(dataset))
idx_tr, idx_va = np.split(idxs, [split])
batch_size = 32

***

#### Here comes the protein test set: 

***

In [ ]:
sequencesTE, NodesTE, nu_samplesTE, labelsTE, C = prep_dataset(IsAcid,pKa_List,AA_List, AA_List_Full,'/content/IPC2_protein_25.csv')

In [ ]:
datasetTE = MyDataset(nu_samplesTE,n_AA=21,Nodes=NodesTE,
                      sequences=sequencesTE,labels=labelsTE,col21_28=col21_28,
                      Just_Descriptors=False,transforms=NormAdj())

***

#### ...and here's the peptide test set. We'll load it and just keep it till we need it. As noted earlier, we won't train on the peptide dataset, but rather turn directly to the combined peptide+protein dataset... later on. 

***

In [ ]:
sequences_pepTE, Nodes_pepTE, nu_samples_pepTE, labels_pepTE, C = prep_dataset(IsAcid,pKa_List,AA_List, AA_List_Full,'/content/IPC2_peptide_25.csv')

In [ ]:
dataset_pepTE = MyDataset(nu_samples_pepTE,n_AA=21,Nodes=Nodes_pepTE,
                      sequences=sequences_pepTE,labels=labels_pepTE,col21_28=col21_28,
                      Just_Descriptors=False,transforms=NormAdj())

***

####  ...Now let's separate the training dataset into the actual training and validation sets, and get the DisjointLoader on board. 

***

In [ ]:
dataset_tr, dataset_va = dataset[idx_tr], dataset[idx_va] #training and validation
loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=1000)
loader_va = DisjointLoader(dataset_va, batch_size=batch_size)
loader_te = DisjointLoader(datasetTE, batch_size=batch_size) #test set
loader_pep_te = DisjointLoader(dataset_pepTE, batch_size=batch_size) #peptide test set

***

In [ ]:
#### The GCN architecture used for this paper is given below, but for brevity, I'll only show the runs with GIN models.

In [ ]:
class GCN(keras.models.Model):
    
    def __init__(self, channels, n_layers, dropout_rate=.1):
        
        super().__init__()
        
        self.conv1 = spk.layers.GCNConv(channels)
        self.convs = []
        
        for _ in range(1, n_layers):
            self.convs.append(
                spk.layers.GCNConv(channels)
            )
        self.pool = spk.layers.GlobalAvgPool()
        self.dense1 = keras.layers.Dense(channels, activation='relu')
        self.dropout = keras.layers.Dropout(dropout_rate)
        self.dense2 = keras.layers.Dense(1)

    def call(self, inputs):
        x, a, i = inputs
        x = self.conv1([x, a])
        for conv in self.convs:
            x = conv([x, a])
        x = self.pool([x, i])
        x = self.dense1(x)
        x = self.dropout(x)
        return self.dense2(x)

In [ ]:
class GIN(keras.models.Model):
    def __init__(self, channels, n_layers,dropout_rate=.1):
        super().__init__()
        self.conv1 = spk.layers.GINConv(channels, epsilon=None, mlp_hidden=[channels, channels],
                                       kernel_initializer=keras.initializers.glorot_uniform(seed=0))
        self.convs = []
        for _ in range(1, n_layers):
            self.convs.append(spk.layers.GINConv(channels, epsilon=None, mlp_hidden=[channels, channels],
                              kernel_initializer=keras.initializers.glorot_uniform(seed=0)))
        self.pool = spk.layers.GlobalAvgPool()
        self.dense1 = keras.layers.Dense(channels, activation="relu",
                                         kernel_initializer=keras.initializers.glorot_uniform(seed=11))
        self.dropout = keras.layers.Dropout(dropout_rate)
        self.dense2 = keras.layers.Dense(1,kernel_initializer=keras.initializers.glorot_uniform(seed=11))

    def call(self, inputs):
        x, a, i = inputs
        x = self.conv1([x, a])
        for conv in self.convs:
            x = conv([x, a])
        x = self.pool([x, i])
        x = self.dense1(x)
        x = self.dropout(x)
        return self.dense2(x)

In [ ]:
loss_fn =  MeanSquaredError()
optimizer = Adam(learning_rate=2e-3)

***

#### The following two cells contain code to run model training. Important points to note: the decoration @tf.function allows to ensure the neural network model is fed with the appropriate input shapes. Run_model checks for the validation loss, and will retrieve the best weights with respect to the validation set. The genetic algorithm–like preselector is presented next (simple_GA). The ouliers calculated by the outlier_calculation function follow the definition used by [2].  

***

In [ ]:
def Define_Run_and_Evaluate(load_tr, mod_name,load_va,load_te,patience_steps,loss_input):
    @tf.function(input_signature=load_tr.tf_signature(), reduce_retracing=True)
    def train_mod_name(inputs,target):
        with tf.GradientTape() as tape:
            predictions = mod_name(inputs, training=True)
            loss = loss_fn(target, predictions) #+ sum(model.losses)
        gradients = tape.gradient(loss, mod_name.trainable_variables)
        optimizer.apply_gradients(zip(gradients, mod_name.trainable_variables))
        return loss

    best_weights, val_losses = Run_model(loader_tr=load_tr,loader_va=load_va,model_input=mod_name,patience_input=patience_steps,
                             loss_input=loss_input,decorated_func = train_mod_name)
    mod_name.set_weights(best_weights)  # Load best model
    test_loss = evaluate(loader=load_te,model_input=mod_name)
    #print(test_loss)
    return best_weights, val_losses, test_loss

In [ ]:
def evaluate(loader,model_input):
    output = []
    step = 0
    while step < loader.steps_per_epoch:
        step += 1
        inputs, target = loader.__next__()
        pred = model_input(inputs, training=False)
        outs = (
            loss_fn(target, pred),
            len(target),  # Keep track of batch size
        )
        output.append(outs)
        if step == loader.steps_per_epoch:
            output = np.array(output)
            return np.average(output[:, :-1], 0, weights=output[:, -1])

def Run_model(loader_tr,loader_va,model_input,patience_input,loss_input,decorated_func):
    epoch = step = 0
    best_val_loss = loss_input
    patience = patience_input
    results = []
    val_losses = []
    for batch in loader_tr:
        step += 1
        #inputs, target = batch
        loss = decorated_func(*batch)
        results.append((loss))
        if step == loader_tr.steps_per_epoch:
            step = 0
            epoch += 1
            if epoch == 1:
                best_weights = model_input.get_weights()

            # Compute validation loss 
            val_loss = evaluate(loader_va,model_input)
            val_losses.append(val_loss)
            # Check if loss improved for early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience = patience_input
                best_weights = model_input.get_weights()
            else:
                patience -= 1
                if patience == 0:
                    break
            results = []

    return best_weights, val_losses

In [ ]:
def simple_GA(opt_weights, y, weights):
    z = len(np.asarray(opt_weights,dtype=object))
    for k in np.arange(z):  
        master_matrix = np.asarray(weights[y[0]][k])
        master_matrix = np.stack((master_matrix,np.asarray(weights[y[1]][k])),axis=0)
        master_matrix1 = np.asarray(weights[y[2]][k])
        master_matrix1 = np.stack((master_matrix1,np.asarray(weights[y[3]][k])),axis=0)
        master_matrix = np.concatenate((master_matrix,master_matrix1),axis=0)

        kth_weights = np.ndarray.flatten(best_weights[k])
        rand_inds = np.ndarray.flatten(np.random.randint(0, 4, size=kth_weights.shape))
        for j in np.arange(len(rand_inds)):
            kth_weights[j] = np.ndarray.flatten(master_matrix[rand_inds[j]])[j]
        opt_weights[k] = kth_weights.reshape(master_matrix[0].shape)
    
    out = opt_weights
    return out

In [ ]:
def outlier_calculation(model_name,loader):
    output = 0
    step = 0
    while step < loader.steps_per_epoch:
        step += 1
        inputs, target = loader.__next__()
        pred = model_name(inputs, training=False)
        output += sum((abs(np.array(target-pred))>.5))
        if step == loader.steps_per_epoch:
            print(output)

***

#### Let's see how well our combined dataset is learned by a four-layer GIN model...

***

In [ ]:
for _ in np.arange(1): #Let's let a single run suffice 
    weights = []
    vals = []
    for _ in np.arange(36):
        model_pro = GIN(channels=8,dropout_rate=.15,n_layers=4)
        best_weights, val_losses, test_loss = Define_Run_and_Evaluate(loader_tr, model_pro,loader_va,
                                                                      loader_te,
                                                                      patience_steps=1,loss_input=0)
        weights.append(best_weights)
        vals.append(val_losses[-1])

    vals = np.asarray(vals)
    y=np.argsort(np.ndarray.flatten(vals))
    opt_weights = simple_GA(best_weights, y, weights)

    z = len(np.asarray(opt_weights,dtype=object))
    model_pro.set_weights(opt_weights) 
    best_weights, val_losses, test_loss = Define_Run_and_Evaluate(loader_tr, model_pro,loader_va,
                                                                  loader_pep_te,
                                                                  patience_steps=40,loss_input=np.Inf)

In [ ]:
test_loss = evaluate(loader=loader_te,model_input=model_pro)
print(test_loss**.5)
test_loss = evaluate(loader=loader_pep_te,model_input=model_pro)
print(test_loss**.5)

In [ ]:
model_pro.save_weights('newer_weights')

In [ ]:
outlier_calculation(model_name=model_pro,loader=loader_te)
outlier_calculation(model_name=model_pro,loader=loader_pep_te)

***

***

### If everything goes to plan, you should get RMSE values of about 0.87–0.88 and 0.27–0.28 on the protein and peptide test sets. Now let's predict a pI for a new AA sequence. Feel free to use my 'original_weights'. 

***

In [ ]:
Your_Pro = [''] #copy and paste AA sequence; please keep it to a single string
#alternatively, read in a FASTA file below
Seq, Nod, Nus, Labs, computed_pI = prep_dataset(IsAcid,pKa_List,AA_List, AA_List_Full,'/content/filename.fasta')
your_dataset = MyDataset(Nus,n_AA=21,Nodes=Nod,sequences=Seq,labels=Labs,
                    col21_28=col21_28,Just_Descriptors=False,
                    transforms=NormAdj())
your_loader = DisjointLoader(your_dataset)
your_model = GIN(channels=8,dropout_rate=.15,n_layers=4)
your_model.load_weights('original_weights')
_, B = evaluate(your_loader,your_model)
MM = computed_pI + B
print(f'The predicted pI is {np.squeeze(MM)}')

### References

#### 1. <a href = "https://chemrxiv.org/engage/chemrxiv/article-details/6399e788ff4651008f2a4e2b"> Brenner 2022 </a>
#### 2. <a href = http://ipc2.mimuw.edu.pl/>Kozlowski 2021 </a>
#### 3. <a href = https://github.com/danielegrattarola/spektral/> Spektral </a>
#### 4. <a href = https://www.wiley.com/en-ie/Solomons%27+Organic+Chemistry,+12th+Edition,+Global+Edition-p-9781119248972> Solomons, Fryhle, & Snyder 2017 </a>
#### 5. <a href = https://journals.aps.org/pre/abstract/10.1103/PhysRevE.75.011920>Moret & Zebende 2007 </a>
#### 6. <a href = https://pubmed.ncbi.nlm.nih.gov/3209351/>Fauchere, Charton, Kier, Verloop & Pliska 1988 </a>
#### 7. <a href = https://www.pnas.org/doi/10.1073/pnas.96.22.12524> Koehl & Levitt, 1999 </a>

***